In [1]:
#!/usr/bin/env python3
import argparse
from importlib import import_module
from itertools import count
import os

import h5py
import json
import numpy as np
from sklearn.metrics import average_precision_score
import tensorflow as tf

import commoncuhk
import loss

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras.utils.np_utils import to_categorical
from keras.layers import LeakyReLU, Concatenate, concatenate, Lambda, UpSampling2D, Add, Input, Dense
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, BatchNormalization

In [ ]:
python cuhk-evaluate.py --excluder cuhk03 
--query_dataset data/cuhk03-np/detected_query.csv 
--query_embeddings experiments/cuhk1rec/query_embeddings.h5 
--gallery_dataset data/cuhk03-np/detected_test.csv 
--gallery_embeddings experiments/cuhk1rec/gallery_embeddings.h5 --metric euclidean --batch_size 18



In [2]:
class Args:
    excluder= 'market1501' #, 'diagonal','cuhk03','duke'
    query_dataset = '/home/k/kajal/triplet-reid/data/cuhk03-np/detected_query.csv'
    query_embeddings = '/home/k/kajal/triplet-reid/experiments/cuhk_res101/query_embeddings.h5'
    query_embeddings_adv = '/home/k/kajal/triplet-reid/experiments/cuhk1rec/query_embeddings.h5'
    gallery_dataset = '/home/k/kajal/triplet-reid/data/cuhk03-np/detected_test.csv'
    gallery_embeddings =  '/home/k/kajal/triplet-reid/experiments/cuhk_res101/gallery_embeddings.h5'
    metric =  'euclidean'
    #filename = 
    batch_size = 18

In [3]:
args=Args()

In [4]:
# Load the query and gallery data from the CSV files.
query_pids, query_fids = commoncuhk.load_dataset(args.query_dataset, None)
gallery_pids, gallery_fids = commoncuhk.load_dataset(args.gallery_dataset, None)

# Load the two datasets fully into memory.
with h5py.File(args.query_embeddings, 'r') as f_query:
    query_embs = np.array(f_query['emb'])
with h5py.File(args.gallery_embeddings, 'r') as f_gallery:
    gallery_embs = np.array(f_gallery['emb'])

In [5]:
gallery_embs=gallery_embs
gallery_pids=gallery_pids

In [6]:
uni_pids=np.unique(gallery_pids)
print(len(uni_pids))
uni_query_pids=np.unique(query_pids)
len(uni_query_pids)

700


700

In [7]:
print(query_embs.shape)
print(gallery_embs.shape)

(1400, 128)
(5332, 128)


In [8]:
query_data = query_pids
query_values = array(query_data)
print('values', query_values)
# integer encode
query_label_encoder = LabelEncoder()
query_integer_encoded = query_label_encoder.fit_transform(query_values)
print('encoded', query_integer_encoded)
# binary encode
query_onehot_encoder = OneHotEncoder(sparse=False)
query_integer_encoded = query_integer_encoded.reshape(len(query_integer_encoded), 1)
query_onehot_encoded = query_onehot_encoder.fit_transform(query_integer_encoded)
print(query_onehot_encoded.shape)

values ['722' '103' '156' ... '369' '374' '1107']
encoded [538  18 234 ... 351 354  53]
(1400, 700)


In [9]:
data = gallery_pids
values = array(data)
print('values', values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print('encoded', integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded.shape)

values ['136' '704' '303' ... '1111' '802' '1199']
encoded [177 527 314 ...  56 586 101]
(5332, 700)


In [10]:
X_train  = gallery_embs
Y_train_label = onehot_encoded
X_test = query_embs
Y_test_label = query_onehot_encoded

In [11]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
#model.add( Dense(units=128, input_dim=128, kernel_initializer='normal', activation='relu') )
#model.add( Dropout(0.5))
model.add( Dense(units=700, input_dim=128, kernel_initializer='normal', activation='softmax') )
print( model.summary() )
model.compile( loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 700)               90300     
                                                                 
Total params: 90,300
Trainable params: 90,300
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
train_history = model.fit(x=gallery_embs, y=Y_train_label, validation_split=0.2, epochs=4, batch_size=128, verbose=2)

Epoch 1/4
34/34 - 0s - loss: 0.2185 - accuracy: 0.9988 - val_loss: 1.6067 - val_accuracy: 0.6729 - 496ms/epoch - 15ms/step
Epoch 2/4
34/34 - 0s - loss: 0.2083 - accuracy: 0.9988 - val_loss: 1.6031 - val_accuracy: 0.6710 - 448ms/epoch - 13ms/step
Epoch 3/4
34/34 - 0s - loss: 0.1987 - accuracy: 0.9991 - val_loss: 1.5950 - val_accuracy: 0.6673 - 418ms/epoch - 12ms/step
Epoch 4/4
34/34 - 0s - loss: 0.1903 - accuracy: 0.9988 - val_loss: 1.5862 - val_accuracy: 0.6673 - 379ms/epoch - 11ms/step


In [29]:
print("Evaluate on test data")
results = model.evaluate(X_test, Y_test_label)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(X_test)
print("predictions shape:", predictions)

Evaluate on test data
44/44 [==============================] - 0s 9ms/step - loss: 0.7076 - accuracy: 0.8829
test loss, test acc: [0.7076228857040405, 0.8828571438789368]
Generate predictions for 3 samples
44/44 [==============================] - 0s 3ms/step
predictions shape: [[2.6948990e-02 5.2166570e-07 3.8133643e-09 ... 3.4685070e-07
  6.9019961e-07 5.7114262e-06]
 [6.3826032e-05 3.3079999e-05 1.3508439e-08 ... 1.1177374e-07
  3.0135901e-08 2.2582599e-05]
 [3.1053105e-03 3.1782231e-06 6.6372266e-08 ... 1.2219591e-06
  9.4686925e-07 2.0365996e-04]
 ...
 [1.6040160e-05 1.9363355e-07 3.2714297e-06 ... 3.7870146e-04
  8.1811057e-05 1.0632847e-04]
 [9.8227232e-05 4.4606049e-06 1.0474688e-08 ... 2.5859282e-07
  8.4168488e-07 3.6592743e-07]
 [2.3667778e-06 9.3013060e-04 4.1372183e-05 ... 5.2756146e-05
  2.2995553e-05 3.2336355e-04]]


In [ ]:
def Hbeta(D=np.array([]), beta=1.0):
    """
        Compute the perplexity and the P-row for a specific value of the
        precision of a Gaussian distribution.
    """

    # Compute P-row and corresponding perplexity
    P = np.exp(-D.copy() * beta)
    sumP = sum(P)
    H = np.log(sumP) + beta * np.sum(D * P) / sumP
    P = P / sumP
    return H, P

def x2p(X=np.array([]), tol=1e-5, perplexity=30.0):
    """
        Performs a binary search to get P-values in such a way that each
        conditional Gaussian has the same perplexity.
    """

    # Initialize some variables
    print("Computing pairwise distances...")
    (n, d) = X.shape
    sum_X = np.sum(np.square(X), 1)
    D = np.add(np.add(-2 * np.dot(X, X.T), sum_X).T, sum_X)
    P = np.zeros((n, n))
    beta = np.ones((n, 1))
    logU = np.log(perplexity)

    # Loop over all datapoints
    for i in range(n):

        # Print progress
        if i % 500 == 0:
            print("Computing P-values for point %d of %d..." % (i, n))

        # Compute the Gaussian kernel and entropy for the current precision
        betamin = -np.inf
        betamax = np.inf
        Di = D[i, np.concatenate((np.r_[0:i], np.r_[i+1:n]))]
        (H, thisP) = Hbeta(Di, beta[i])

        # Evaluate whether the perplexity is within tolerance
        Hdiff = H - logU
        tries = 0
        while np.abs(Hdiff) > tol and tries < 50:

            # If not, increase or decrease precision
            if Hdiff > 0:
                betamin = beta[i].copy()
                if betamax == np.inf or betamax == -np.inf:
                    beta[i] = beta[i] * 2.
                else:
                    beta[i] = (beta[i] + betamax) / 2.
            else:
                betamax = beta[i].copy()
                if betamin == np.inf or betamin == -np.inf:
                    beta[i] = beta[i] / 2.
                else:
                    beta[i] = (beta[i] + betamin) / 2.

            # Recompute the values
            (H, thisP) = Hbeta(Di, beta[i])
            Hdiff = H - logU
            tries += 1

        # Set the final row of P
        P[i, np.concatenate((np.r_[0:i], np.r_[i+1:n]))] = thisP

    # Return final P-matrix
    print("Mean value of sigma: %f" % np.mean(np.sqrt(1 / beta)))
    return P

def pca(X=np.array([]), no_dims=50):
    """
        Runs PCA on the NxD array X in order to reduce its dimensionality to
        no_dims dimensions.
    """

    print("Preprocessing the data using PCA...")
    (n, d) = X.shape
    X = X - np.tile(np.mean(X, 0), (n, 1))
    (l, M) = np.linalg.eig(np.dot(X.T, X))
    Y = np.dot(X, M[:, 0:no_dims])
    return Y
def tsne(X=np.array([]), no_dims=2, initial_dims=50, perplexity=30.0):
    """
        Runs t-SNE on the dataset in the NxD array X to reduce its
        dimensionality to no_dims dimensions. The syntaxis of the function is
        `Y = tsne.tsne(X, no_dims, perplexity), where X is an NxD NumPy array.
    """

    # Check inputs
    if isinstance(no_dims, float):
        print("Error: array X should have type float.")
        return -1
    if round(no_dims) != no_dims:
        print("Error: number of dimensions should be an integer.")
        return -1

    # Initialize variables
    X = pca(X, initial_dims).real
    (n, d) = X.shape
    max_iter = 10
    initial_momentum = 0.5
    final_momentum = 0.8
    eta = 200
    min_gain = 0.01
    Y = np.random.randn(n, no_dims)
    dY = np.zeros((n, no_dims))
    iY = np.zeros((n, no_dims))
    gains = np.ones((n, no_dims))

    # Compute P-values
    P = x2p(X, 1e-5, perplexity)
    P = P + np.transpose(P)
    P = P / np.sum(P)
    P = P * 4.									# early exaggeration
    P = np.maximum(P, 1e-12)
    

    # Run iterations
    for iter in range(max_iter):

        # Compute pairwise affinities
        sum_Y = np.sum(np.square(Y), 1)
        num = -2. * np.dot(Y, Y.T)
        num = 1. / (1. + np.add(np.add(num, sum_Y).T, sum_Y))
        num[range(n), range(n)] = 0.
        Q = num / np.sum(num)
        Q = np.maximum(Q, 1e-12)

        # Compute gradient
        PQ = P - Q
        for i in range(n):
            dY[i, :] = np.sum(np.tile(PQ[:, i] * num[:, i], (no_dims, 1)).T * (Y[i, :] - Y), 0)

        # Perform the update
        if iter < 20:
            momentum = initial_momentum
        else:
            momentum = final_momentum
        gains = (gains + 0.2) * ((dY > 0.) != (iY > 0.)) + \
                (gains * 0.8) * ((dY > 0.) == (iY > 0.))
        gains[gains < min_gain] = min_gain
        iY = momentum * iY - eta * (gains * dY)
        Y = Y + iY
        Y = Y - np.tile(np.mean(Y, 0), (n, 1))

        # Compute current value of cost function
        if (iter + 1) % 10 == 0:
            C = np.sum(P * np.log(P / Q))
            print("Iteration %d: error is %f" % (iter + 1, C))

        # Stop lying about P-values
        if iter == 100:
            P = P / 4.

    # Return solution
    return Y
             

In [ ]:
import pylab
X = gallery_embs[500:700]
lbl=list(map(int,gallery_pids))
labels = lbl[500:700]#query_pids[0:20]
pylab.figure()
Y = tsne(X, 2, 50, 20.0)
plt=pylab.scatter(Y[:, 0],Y[:,1],10,labels,marker='o')
pylab.axis('off')
pylab.savefig('rsltn3n2.jpg')
pylab.show()

In [ ]:
import pylab
X = query_embs[500:700]
lbl=list(map(int,query_pids))
labels = lbl[500:700]#query_pids[0:20]
pylab.figure()
Y = tsne(X, 2, 50, 20.0)
plt=pylab.scatter(Y[:, 0],Y[:,1],10,labels,marker='o')
pylab.axis('off')
pylab.savefig('rsltn3n1.jpg')
pylab.show()

In [ ]:
from diffprivlib.models import GaussianNB
clf = GaussianNB(epsilon=0.01)
clf.fit(X_test, query_pids)

In [ ]:
clf.predict(X_test)

In [ ]:
print("Test accuracy: %f" % clf.score(X_test, query_pids))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

epsilons = np.logspace(-2, 2, 50)
#bounds = ([4.3, 2.0, 1.1, 0.1], [7.9, 4.4, 6.9, 2.5])
bounds=(np.array(128),np.array(128))
accuracy = list()

for epsilon in epsilons:
    clf = GaussianNB(bounds=bounds, epsilon=epsilon)
    clf.fit(X_train, gallery_pids)
    
    accuracy.append(clf.score(X_test, query_pids))

plt.semilogx(epsilons, accuracy)
plt.title("Differentially private Naive Bayes accuracy")
plt.xlabel("epsilon")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
import diffprivlib.models as dp
import numpy as np
from sklearn.linear_model import LogisticRegression
dp_clf = dp.LogisticRegression()
dp_clf.fit(X_train, gallery_pids)

In [ ]:
print("Differentially private test accuracy (epsilon=%.2f): %.2f%%" % 
     (dp_clf.epsilon, dp_clf.score(X_test, query_pids) * 100))

In [ ]:
import sklearn as sk
from sklearn.pipeline import Pipeline

from diffprivlib import models
pipe = Pipeline([
    ('scaler', sk.preprocessing.StandardScaler()),
    ('pca', sk.decomposition.PCA(2)),
    ('lr', sk.linear_model.LogisticRegression(solver="lbfgs"))
])
pipe.fit(X_train, gallery_pids)
baseline = pipe.score(X_test, query_pids)
print("Non-private test accuracy: %.2f%%" % (baseline * 100))

In [ ]:
dp_pipe = Pipeline([
    ('scaler', models.StandardScaler(bounds=(np.array(128), np.array(128)))),
    ('pca', models.PCA(2, data_norm=5, centered=True)),
    ('lr', models.LogisticRegression(data_norm=5))
])

dp_pipe.fit(X_train, gallery_pids)
print("Differentially private pipeline accuracy (epsilon=3): %.2f%%" % (dp_pipe.score(X_test, query_pids) * 100))